In [1]:
from bs4 import BeautifulSoup as Soup
import pandas as pd
import re
import requests

In [2]:
url = "https://govisity.com/most-visited-countries-world/"
link = requests.get(url)
parsed_content = Soup(link.content, 'html.parser')

In [3]:
parsed_content

<!DOCTYPE html>

<html class="no-js" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<link href="" rel="pingback"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<!-- Social Warfare v4.4.1 https://warfareplugins.com - BEGINNING OF OUTPUT -->
<style>
@font-face {
font-family: "sw-icon-font";
src:url("https://govisity.com/wp-content/plugins/social-warfare/assets/fonts/sw-icon-font.eot?ver=4.4.1");
src:url("https://govisity.com/wp-content/plugins/social-warfare/assets/fonts/sw-icon-font.eot?ver=4.4.1#iefix") format("embedded-opentype"),
url("https://govisity.com/wp-content/plugins/social-warfare/assets/fonts/sw-icon-font.woff?ver=4.4.1") format("woff"),
url("https://govisity.com/wp-content/plugins/social-warfare/assets/fonts/sw-icon-font.ttf?ver=4.4.1") format("truetype"),
url("https://govisity.com/wp-conten

In [4]:
part1 = parsed_content.find("ul", {"class": "ez-toc-list ez-toc-list-level-1"})
part2 = part1.find_all('li')
part2 = part2[1:]
part2

[<li class="ez-toc-heading-level-3"><a class="ez-toc-link ez-toc-heading-2" href="#1_France_%E2%80%93_826_million_visitors" title="1. France – 82.6 million visitors">1. France – 82.6 million visitors</a></li>,
 <li class="ez-toc-page-1 ez-toc-heading-level-3"><a class="ez-toc-link ez-toc-heading-3" href="#2_The_United_States_%E2%80%93_756_million_visitors" title="2. The United States – 75.6 million visitors">2. The United States – 75.6 million visitors</a></li>,
 <li class="ez-toc-page-1 ez-toc-heading-level-3"><a class="ez-toc-link ez-toc-heading-4" href="#3_Spain_%E2%80%93_756_million_visitors" title="3. Spain – 75.6 million visitors">3. Spain – 75.6 million visitors</a></li>,
 <li class="ez-toc-page-1 ez-toc-heading-level-3"><a class="ez-toc-link ez-toc-heading-5" href="#4_China_%E2%80%93_593_million_visitors" title="4. China – 59.3 million visitors">4. China – 59.3 million visitors</a></li>,
 <li class="ez-toc-page-1 ez-toc-heading-level-3"><a class="ez-toc-link ez-toc-heading-6" h

In [5]:
items = [li.a['href'] for li in part2]
items

['#1_France_%E2%80%93_826_million_visitors',
 '#2_The_United_States_%E2%80%93_756_million_visitors',
 '#3_Spain_%E2%80%93_756_million_visitors',
 '#4_China_%E2%80%93_593_million_visitors',
 '#5_Italy_%E2%80%93_524_million_visitors',
 'https://govisity.com/most-visited-countries-world/2/#6_United_Kingdom_%E2%80%93_358_million_visitors',
 'https://govisity.com/most-visited-countries-world/2/#7_Germany_%E2%80%93_356_million_visitors',
 'https://govisity.com/most-visited-countries-world/2/#8_Mexico_%E2%80%93_350_million_visitors',
 'https://govisity.com/most-visited-countries-world/2/#9_Thailand_%E2%80%93_326_million_visitors',
 'https://govisity.com/most-visited-countries-world/2/#10_Turkey_%E2%80%93_30_million_visitors',
 'https://govisity.com/most-visited-countries-world/3/#11_Austria_%E2%80%93_281_million_visitors',
 'https://govisity.com/most-visited-countries-world/3/#12_Malaysia_%E2%80%93_268_million_visitors',
 'https://govisity.com/most-visited-countries-world/3/#13_Hong_Kong_%E2%

In [6]:
links = [items[i] for i in range(0, len(items), 5)]
links

['#1_France_%E2%80%93_826_million_visitors',
 'https://govisity.com/most-visited-countries-world/2/#6_United_Kingdom_%E2%80%93_358_million_visitors',
 'https://govisity.com/most-visited-countries-world/3/#11_Austria_%E2%80%93_281_million_visitors',
 'https://govisity.com/most-visited-countries-world/4/#16_Japan_%E2%80%93_240_million_visitors',
 'https://govisity.com/most-visited-countries-world/5/#21_Netherlands_%E2%80%93_158_million_visitors',
 'https://govisity.com/most-visited-countries-world/6/#26_Croatia_%E2%80%93_138_million_visitors']

In [7]:
df = pd.DataFrame(columns = ["CountryName", "Visitors", "1", "2", "3", "4", "5"])

In [8]:
for i in range(len(links)):
    if i == 0:
        url3 = f"{url}{links[i]}"
        response2 = requests.get(url3)
    else:
        url3 = links[i]
        response2 = requests.get(url3)
    parsed_data = Soup(response2.content, 'html.parser')
    blockquotes = parsed_data.find_all("blockquote")
    itemlist = []
    for blockquote in blockquotes:
        if blockquote.h4 == None:
            continue
        else:
            itemlist.append(blockquote.p)
    strong_tags = []
    for html in itemlist:
        for strong_tag in html.find_all('strong'):
            if strong_tag.find('a'):
                continue
            else: strong_tags.append(strong_tag)
    places = [strong.getText() for strong in strong_tags]
    for j in range(0, len(places), 5):
        df.loc[len(df)] = ["NA", "NA", places[j], 
                           places[j + 1], places[j + 2], places[j + 3], places[j + 4]]

In [9]:
names = [li.a.getText().split(" – ")[0] for li in part2]
names

['1. France',
 '2. The United States',
 '3. Spain',
 '4. China',
 '5. Italy',
 '6. United Kingdom',
 '7. Germany',
 '8. Mexico',
 '9. Thailand',
 '10. Turkey',
 '11. Austria',
 '12. Malaysia',
 '13. Hong Kong',
 '14. Greece',
 '15. Russia',
 '16. Japan',
 '17. Canada',
 '18. Saudi Arabia',
 '19. Poland',
 '20. South Korea',
 '21. Netherlands',
 '22. Macao',
 '23. Hungary',
 '24. United Arab Emirates',
 '25. India',
 '26. Croatia',
 '27. Ukraine',
 '28. Singapore',
 '29. Indonesia',
 '30. Czech Republic']

In [10]:
country_names = [name.split(". ")[1] for name in names]
country_names

['France',
 'The United States',
 'Spain',
 'China',
 'Italy',
 'United Kingdom',
 'Germany',
 'Mexico',
 'Thailand',
 'Turkey',
 'Austria',
 'Malaysia',
 'Hong Kong',
 'Greece',
 'Russia',
 'Japan',
 'Canada',
 'Saudi Arabia',
 'Poland',
 'South Korea',
 'Netherlands',
 'Macao',
 'Hungary',
 'United Arab Emirates',
 'India',
 'Croatia',
 'Ukraine',
 'Singapore',
 'Indonesia',
 'Czech Republic']

In [11]:
visitors = [li.a.getText().split(" – ")[1] for li in part2]
visitors

['82.6 million visitors',
 '75.6 million visitors',
 '75.6 million visitors',
 '59.3 million visitors',
 '52.4 million visitors',
 '35.8 million visitors',
 '35.6 million visitors',
 '35.0 million visitors',
 '32.6 million visitors',
 '30 million visitors',
 '28.1 million visitors',
 '26.8 million visitors',
 '26.6 million visitors',
 '24.8 million visitors',
 '24.6 million visitors',
 '24.0 million visitors',
 '20.0 million visitors',
 '18.0 million visitors',
 '17.5 million visitors',
 '17.2 million visitors',
 '15.8 million visitors',
 '15.7 million visitors',
 '15.3 million visitors',
 '14.9 million visitors',
 '14.6 million visitors',
 '13.8 million visitors',
 '13.3 million visitors',
 '12.9 million visitors',
 '12.0 million visitors',
 '11.9 million visitors']

In [12]:
df['CountryName'] = country_names
df['Visitors'] = visitors
df.head()

,CountryName,Visitors,1,2,3,4,5
0,France,82.6 million visitors,Eiffel tower,St Tropez,Chamonix,Palace of Versailles,Mont Saint-Michel
1,The United States,75.6 million visitors,Grand Canyon,Manhattan,Yellowstone National Park,Golden Gate Bridge,Niagara Falls
2,Spain,75.6 million visitors,Alhambra,Mezquita of Cordoba,El Escorial,Sagrada Familia in Barcelona,Ibiza island
3,China,59.3 million visitors,The Great Wall of China,The Terracotta Army,The Forbidden City,The Li River,The Yellow Mountains
4,Italy,52.4 million visitors,Colosseum in Rome,Grand Canal in Venice,Florence Cathedral,Piazza del Campo in Siena,Pompeii
